# **벤처 기업의 매출 규모 예측**
(2020년)

데이터링크
https://mdis.kostat.go.kr/ofrData/selectOfrDataDetail.do?survId=14203&itmDiv=2&nPage=3&itemId=2010&itemNm=%EA%B2%BD%EC%A0%9C%EC%9D%BC%EB%B0%98%C2%B7%EA%B2%BD%EA%B8%B0/%EA%B8%B0%EC%97%85%EA%B2%BD%EC%98%81#click

## 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import axes3d

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 155632 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/계량경제/2020_총괄_20220606_03411.csv', encoding='cp949')
df.head()

,[쿼터] 규모,A2-2. 기업성장단계,B2-8. 창업 당시 창업자 실무경험 년수(년),C1-1. 2019년 정규직_합계,C3. 학력별 인력구성_대졸,D1. 지분구조(%)_창업자,D2-2. 2019년 부채(백만원),E2-1. 현재 등록보유 산업재산권(건)_합계,G3-1. 주력제품 국내 시장점유율(%)
0,1,1,0.0,2,1,67.0,156,5,2.00
1,1,1,10.0,19,16,26.0,7771,3,1.00
2,1,1,10.0,2,0,NaN,318,1,NaN
3,1,1,10.0,2,2,90.0,378,17,0.01
4,1,1,10.0,4,3,NaN,155,1,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   [쿼터] 규모                     2500 non-null   int64  
 1   A2-2. 기업성장단계                2500 non-null   int64  
 2   B2-8. 창업 당시 창업자 실무경험 년수(년)  2500 non-null   float64
 3   C1-1. 2019년 정규직_합계          2500 non-null   int64  
 4   C3. 학력별 인력구성_대졸             2500 non-null   int64  
 5   D1. 지분구조(%)_창업자             2200 non-null   float64
 6   D2-2. 2019년 부채(백만원)         2500 non-null   int64  
 7   E2-1. 현재 등록보유 산업재산권(건)_합계   2500 non-null   int64  
 8   G3-1. 주력제품 국내 시장점유율(%)      2200 non-null   float64
dtypes: float64(3), int64(6)
memory usage: 175.9 KB


총 2500개의 데이터

In [ ]:
df.columns = ['규모', '기업성장단계','실무년수', '정규직', '대졸인력', '지분구조', '부채', '산업재산권', '시장점유율']

In [ ]:
df.head()

,규모,기업성장단계,실무년수,정규직,대졸인력,지분구조,부채,산업재산권,시장점유율
0,1,1,0.0,2,1,67.0,156,5,2.00
1,1,1,10.0,19,16,26.0,7771,3,1.00
2,1,1,10.0,2,0,NaN,318,1,NaN
3,1,1,10.0,2,2,90.0,378,17,0.01
4,1,1,10.0,4,3,NaN,155,1,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   규모      2500 non-null   int64  
 1   기업성장단계  2500 non-null   int64  
 2   실무년수    2500 non-null   float64
 3   정규직     2500 non-null   int64  
 4   대졸인력    2500 non-null   int64  
 5   지분구조    2200 non-null   float64
 6   부채      2500 non-null   int64  
 7   산업재산권   2500 non-null   int64  
 8   시장점유율   2200 non-null   float64
dtypes: float64(3), int64(6)
memory usage: 175.9 KB


## 전처리 
결측치 median으로

In [ ]:
df['지분구조'].fillna((df['지분구조'].median()), inplace=True)

In [ ]:
df['시장점유율'].fillna((df['시장점유율'].median()), inplace=True)

In [ ]:
df.isna().sum().sum()

0

In [ ]:
df["규모"] = df["규모"].replace(1,1)
df["규모"] = df["규모"].replace(2,5)
df["규모"] = df["규모"].replace(3,20)
df["규모"] = df["규모"].replace(4,50)
df["규모"] = df["규모"].replace(5,80)
df["규모"] = df["규모"].replace(6,120)
df["규모"] = df["규모"].replace(7,200)
df["규모"] = df["규모"].replace(8,500)

In [ ]:
df['규모'].value_counts() # y 매출규모

80     661
20     406
200    334
50     316
1      286
120    277
500    220
Name: 규모, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   규모      2500 non-null   int64  
 1   기업성장단계  2500 non-null   int64  
 2   실무년수    2500 non-null   float64
 3   정규직     2500 non-null   int64  
 4   대졸인력    2500 non-null   int64  
 5   지분구조    2500 non-null   float64
 6   부채      2500 non-null   int64  
 7   산업재산권   2500 non-null   int64  
 8   시장점유율   2500 non-null   float64
dtypes: float64(3), int64(6)
memory usage: 175.9 KB


In [ ]:
df.describe()

,규모,기업성장단계,실무년수,정규직,대졸인력,지분구조,부채,산업재산권,시장점유율
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,114.850400,3.261200,10.531960,47.598800,21.310800,56.468720,8755.375200,11.918800,4.770828
std,133.526296,0.824892,6.982106,64.395016,32.537438,29.362204,18476.688633,23.439709,9.256763
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,3.000000,5.000000,10.000000,4.000000,40.000000,933.000000,1.000000,0.100000
50%,80.000000,3.000000,10.000000,23.000000,10.000000,55.000000,3008.500000,4.000000,1.000000
75%,120.000000,4.000000,15.000000,57.000000,24.000000,80.000000,7883.000000,12.000000,5.000000
max,500.000000,5.000000,40.000000,658.000000,490.000000,100.000000,380473.000000,230.000000,80.000000


## 선형 모델링

### Simple Linear Regression

In [ ]:
  # simple linear regressor as reference
X = df['정규직'] # student teacher ratio
y = df['규모'] # test score

X = sm.add_constant(X) # adding a constant

model = sm.OLS(y, X).fit(cov_type='HC0')

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                     규모   R-squared:                       0.459
Model:                            OLS   Adj. R-squared:                  0.458
Method:                 Least Squares   F-statistic:                     438.4
Date:                Mon, 06 Jun 2022   Prob (F-statistic):           8.18e-90
Time:                        15:06:46   Log-Likelihood:                -15016.
No. Observations:                2500   AIC:                         3.004e+04
Df Residuals:                    2498   BIC:                         3.005e+04
Df Model:                           1                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         48.0107      2.878     16.683      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


- R-squared:                       0.459

### Multiple Linear Regression

insiginficant 한 변수 삭제하는 방법 사용

reject 하면 impact가 있는 거임

In [ ]:
df.corr()

,규모,기업성장단계,실무년수,정규직,대졸인력,지분구조,부채,산업재산권,시장점유율
규모,1.000000,0.209011,-0.062053,0.677211,0.543602,-0.276463,0.597278,0.344966,0.207235
기업성장단계,0.209011,1.000000,-0.077140,0.209930,0.149435,-0.166323,0.151444,0.172377,0.017568
실무년수,-0.062053,-0.077140,1.000000,-0.061976,-0.095243,0.001093,-0.026302,-0.044968,0.048657
정규직,0.677211,0.209930,-0.061976,1.000000,0.832971,-0.301760,0.606100,0.374682,0.194368
대졸인력,0.543602,0.149435,-0.095243,0.832971,1.000000,-0.255811,0.440795,0.251024,0.079365
지분구조,-0.276463,-0.166323,0.001093,-0.301760,-0.255811,1.000000,-0.234827,-0.206682,-0.080373
부채,0.597278,0.151444,-0.026302,0.606100,0.440795,-0.234827,1.000000,0.335721,0.179995
산업재산권,0.344966,0.172377,-0.044968,0.374682,0.251024,-0.206682,0.335721,1.000000,0.164406
시장점유율,0.207235,0.017568,0.048657,0.194368,0.079365,-0.080373,0.179995,0.164406,1.000000


In [ ]:
# 모든 변수
robust_est = smf.ols('규모 ~ 실무년수+정규직+대졸인력+산업재산권+기업성장단계+부채+지분구조+시장점유율', df).fit(cov_type='HC0')
print(robust_est.summary())

                            OLS Regression Results                            
Dep. Variable:                     규모   R-squared:                       0.527
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     115.1
Date:                Mon, 06 Jun 2022   Prob (F-statistic):          4.92e-164
Time:                        15:07:02   Log-Likelihood:                -14847.
No. Observations:                2500   AIC:                         2.971e+04
Df Residuals:                    2491   BIC:                         2.976e+04
Df Model:                           8                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     34.9885      9.298      3.763      0.0

In [ ]:
# 대졸 제외
robust_est = smf.ols('규모 ~ 실무년수+정규직+산업재산권+기업성장단계+부채+지분구조+시장점유율', df).fit(cov_type='HC0')
print(robust_est.summary())

                            OLS Regression Results                            
Dep. Variable:                     규모   R-squared:                       0.527
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     131.9
Date:                Mon, 06 Jun 2022   Prob (F-statistic):          1.62e-165
Time:                        15:07:05   Log-Likelihood:                -14847.
No. Observations:                2500   AIC:                         2.971e+04
Df Residuals:                    2492   BIC:                         2.976e+04
Df Model:                           7                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     35.6815      9.056      3.940      0.0

In [ ]:
# 대졸, 실무년수 제외
robust_est = smf.ols('규모 ~ 정규직+산업재산권+기업성장단계+부채+지분구조+시장점유율', df).fit(cov_type='HC0')
print(robust_est.summary())

                            OLS Regression Results                            
Dep. Variable:                     규모   R-squared:                       0.526
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     152.4
Date:                Mon, 06 Jun 2022   Prob (F-statistic):          4.25e-165
Time:                        15:07:37   Log-Likelihood:                -14849.
No. Observations:                2500   AIC:                         2.971e+04
Df Residuals:                    2493   BIC:                         2.975e+04
Df Model:                           6                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     29.9727      8.480      3.535      0.0

우리 최종 선형모델

**규모 ~ 정규직+산업재산권+기업성장단계+부채+지분구조+시장점유율**

- Adj. R-squared:                  0.525

## 비선형 모델링

### Nonlinear functions of two variables : interaction

Interactions Between Independent Variables: two **continuous regressors**

(ch8 50페이지 참고)

변수간의 correlation이 있을 수 있기 때문에 interaction term 고려

* 대졸이 많으면 정규직이 많지 않을까?

   -> 대졸*정규직 

* 기업성장단계가 크면 시장점유율이 크지 않을까?

   ->기업성장단계*시장점유율 

In [ ]:
robust_est = smf.ols('규모 ~ 실무년수+정규직+대졸인력+산업재산권+기업성장단계+부채+지분구조+시장점유율+정규직*대졸인력+기업성장단계*시장점유율', df).fit(cov_type='HC0')
print(robust_est.summary())

                            OLS Regression Results                            
Dep. Variable:                     규모   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     143.7
Date:                Mon, 06 Jun 2022   Prob (F-statistic):          7.62e-238
Time:                        15:07:41   Log-Likelihood:                -14798.
No. Observations:                2500   AIC:                         2.962e+04
Df Residuals:                    2489   BIC:                         2.968e+04
Df Model:                          10                                         
Covariance Type:                  HC0                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       23.8481      9.207      2.590   

시장점유율과 기업성장단계는 interaction 없었다.

대졸과 정규직은 interaction 있었다.

In [ ]:
#시장점유율*기업성장단계 삭제
robust_est = smf.ols('규모 ~ 실무년수+정규직+대졸인력+산업재산권+기업성장단계+부채+지분구조+시장점유율+정규직*대졸인력', df).fit(cov_type='HC0')
print(robust_est.summary())

                            OLS Regression Results                            
Dep. Variable:                     규모   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     159.0
Date:                Mon, 06 Jun 2022   Prob (F-statistic):          5.85e-238
Time:                        15:07:44   Log-Likelihood:                -14798.
No. Observations:                2500   AIC:                         2.962e+04
Df Residuals:                    2490   BIC:                         2.967e+04
Df Model:                           9                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     21.6419      9.063      2.388      0.0

In [ ]:
# 실무년수 삭제
robust_est = smf.ols('규모 ~ 정규직+대졸인력+산업재산권+기업성장단계+부채+지분구조+시장점유율+정규직*대졸인력', df).fit(cov_type='HC0')
print(robust_est.summary())

                            OLS Regression Results                            
Dep. Variable:                     규모   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     178.9
Date:                Mon, 06 Jun 2022   Prob (F-statistic):          4.85e-239
Time:                        15:07:45   Log-Likelihood:                -14799.
No. Observations:                2500   AIC:                         2.962e+04
Df Residuals:                    2491   BIC:                         2.967e+04
Df Model:                           8                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.0100      8.435      2.254      0.0

In [ ]:
# 산업재산권 삭제
robust_est = smf.ols('규모 ~ 정규직+대졸인력+기업성장단계+부채+지분구조+시장점유율+정규직*대졸인력', df).fit(cov_type='HC0')
print(robust_est.summary())

                            OLS Regression Results                            
Dep. Variable:                     규모   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.542
Method:                 Least Squares   F-statistic:                     199.2
Date:                Mon, 06 Jun 2022   Prob (F-statistic):          4.66e-235
Time:                        15:08:53   Log-Likelihood:                -14802.
No. Observations:                2500   AIC:                         2.962e+04
Df Residuals:                    2492   BIC:                         2.967e+04
Df Model:                           7                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.1178      8.460      2.260      0.0

우리의 최종 비선형 모델은 

규모 ~ 정규직+대졸인력+기업성장단계+부채+지분구조+시장점유율+정규직*대졸인력 
- Adj. R-squared:                  0.542